<a href="https://colab.research.google.com/github/RaduSima/SSL_Project2024/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install datasets
# ! pip install -U accelerate
# ! pip install -U transformers

import itertools

import pandas as pd
import torch
import accelerate
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from time import time as tm


import gc
import numpy
import pickle as pkl
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
class OrdinalRegressionHead(torch.nn.Module):
    """
    Ordinal regression head for classification problems. The way it works is by having a single layer that outputs a single value, which is then added to a learnable bias. The output is then passed through a sigmoid function.
    The bias is a learnable parameter that is used to shift the output of the single layer to the desired range. It will be learned to have descending order.

    class = sum(b_i > 0.5) + 1

    class 1: 0, 0, 0, 0
    class 2: 1, 0, 0, 0
    class 3: 1, 1, 0, 0
    class 4: 1, 1, 1, 0
    class 5: 1, 1, 1, 1
    and so on ...

    TODO: should have more than one fc layer.
    """
    def __init__(self, in_features, num_classes, intermediate_layers=None):
        """
        The constructor for OrdinalRegressionHead class.

        Parameters
        ----------
        in_features : int
            The number of input features. This is the number of features of the output layer of the big model.
        num_classes : int
            The number of classes in the classification problem.
        """
        super(OrdinalRegressionHead, self).__init__()

        if intermediate_layers is None:
            intermediate_layers = []

        input_size = in_features
        layers = []
        for layer_size in intermediate_layers:
            layers.append(torch.nn.Linear(input_size, layer_size))
            layers.append(torch.nn.ReLU())
            input_size = layer_size
        layers.append(torch.nn.Linear(input_size, 1, bias=False))

        self.fc = torch.nn.Sequential(*layers)

        self.b = torch.nn.Parameter(torch.zeros(num_classes - 1))
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        """
        The forward method for OrdinalRegressionHead class.

        Parameters
        ----------
        x : tensor
            the input tensor.

        Returns
        -------
        tuple(tensor, tensor)
            The logits and the output of the model.
            The logits are useful in the training phase, as BCEWithLogitsLoss is used.
        """
        x = self.fc(x)
        y = x + self.b
        return y, self.activation(y)

class OrdinalRegressionClassifier(torch.nn.Module):
    def __init__(self, embeddings_size, num_classes, intermediate_layers=None) -> None:
        super(OrdinalRegressionClassifier, self).__init__()
        self.head = OrdinalRegressionHead(embeddings_size, num_classes, intermediate_layers)
        self.loss = torch.nn.BCEWithLogitsLoss()

    def forward(self, embeddings, labels):
        logits, output = self.head(embeddings)
        return self.loss(logits, labels), output


In [ ]:
class OurClassifierModel(torch.nn.Module):
    """
    OurClassifierModel class is a class for the model that uses the BigBird model and the OrdinalRegressionHead.
    It uses the pretrained classifier of a BigBird model and adds an ordinal regression head on top of it.

    """
    def __init__(self, transformer, embedding_size=768, num_classes=5, intermediate_layers=None):
        super(OurClassifierModel, self).__init__()

        self.transformer = transformer
        # The output of the transformer model is 768, as it is the output of the last hidden state.
        self.classifier = OrdinalRegressionHead(embedding_size, num_classes, intermediate_layers=intermediate_layers)

        self.loss = torch.nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask, labels):
        """
        The forward method for OurClassifierModel class.

        Parameters
        ----------
        input_ids : tensor
            The input tensor, used for transformer.
        attention_mask : tensor
            The attention mask tensor, used for transformer.
        labels : tensor
            The target labels for the classification problem.

        Returns
        -------
        tuple(tensor, tensor)
            The loss and the output of the model.
            The loss is useful in the training phase, as we are using Trainer from HuggingFace.
        """
        x = self.transformer(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        logits, output = self.classifier(x)
        return self.loss(logits, labels), output

class EmbeddingBigBirdModel(torch.nn.Module):
    def __init__(self, bert):
        super(EmbeddingBigBirdModel, self).__init__()
        self.bert = bert
        # The output of the bert model is 768, as it is the output of the last hidden state.
        return

    def forward(self, input_ids, attention_mask):
        """
        The forward method for OurBigBirdModel class.

        Parameters
        ----------
        input_ids : tensor
            The input tensor, used for bert.
        attention_mask : tensor
            The attention mask tensor, used for bert.
        labels : tensor
            The target labels for the classification problem.

        Returns
        -------
        tuple(tensor, tensor)
            The loss and the output of the model.
            The loss is useful in the training phase, as we are using Trainer from HuggingFace.
        """
        return self.bert(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]

In [ ]:
from transformers import BigBirdForSequenceClassification, BigBirdTokenizer, T5ForSequenceClassification, T5Tokenizer
transformers_repo = [
  "google/bigbird-roberta-base",
  "google/bigbird-roberta-large",
  "albert-base-v2",
  "albert-large-v2",
  "allenai/longformer-base-4096",
  "t5-small",
  "t5-base",
  "t5-large",
  "facebook/bart-base",
  "facebook/bart-large",
  "microsoft/deberta-base",
  "microsoft/deberta-large",
]

transformers_classes = {
  "google/bigbird-roberta-base": {
    "model_class": BigBirdForSequenceClassification,
    "tokenizer_class": BigBirdTokenizer,
    "embedding_size": 768,
    },
  "t5-small": {
    "model_class": T5ForSequenceClassification,
    "tokenizer_class": T5Tokenizer,
    "embedding_size": 512,
    },
  "t5-base": {
    "model_class": T5ForSequenceClassification,
    "tokenizer_class": T5Tokenizer,
    "embedding_size": 512,
    },
  "t5-large": {
    "model_class": T5ForSequenceClassification,
    "tokenizer_class": T5Tokenizer,
    "embedding_size": 512,
    },
}

In [ ]:
import os


def remove_percentage(df, percent):
    """
    Useful method for development purposes. It removes a percentage of the rows from the dataframe.

    Parameters
    ----------
    df : dataframe
        The dataframe to remove rows from.
    percent : float
        The percentage of rows to remove.

    Returns
    -------
    dataframe
        The dataframe with the rows removed.
    """
    if percent == 0:
        return df
    numpy.random.seed(42)
    num_rows_to_remove = int(len(df) * percent)
    df_removed = df.sample(frac=1).iloc[num_rows_to_remove:]
    return df_removed


def convert_label_to_one_hot_encodings(labels: list[float], num_classes, max_label=3500):
    """
    Convert the labels to a class representation. The class representation is a one-hot encoding of the labels.
    This preparation is for the ordinal regression problem.

    If the one-hot encoding of the label 3 for multi-class classification is [0, 0, 1, 0, 0],
      then the one-hot encoding of label 3 for ordinal regression is [1, 1, 0, 0].

    To convert from multi-class to ordinal regression, we can do the following trick:
        - compute the one-hot encoding of the labels for multi-class classification
        - replace all the 0s with 1s until the first 1 is found (for multi-class classification a 1 is found at the index of the class)
        - remove the first column of the one-hot encoding (the first 1)

    Parameters
    ----------
    labels : list[float]
        The list of labels to convert.
    num_classes : int
        The number of classes in the classification problem.
    max_label : int, optional
        The maximum label in the dataset. This is used to normalize the labels. The default is 2800.

    Returns
    -------
    tensor
        The tensor of the one-hot encoding of the labels.
    """
    # reminder: in ordinal regression, class = sum(output > 0.5) + 1

    # normalize the labels
    labels = numpy.array(labels) / max_label
    class_labels = numpy.zeros((len(labels), num_classes - 1))
    for i, label in enumerate(labels):
        class_labels[i] = numpy.array(
            [1 if j / num_classes <= label else 0 for j in range(1, num_classes)])
    return torch.tensor(class_labels)


def compute_metrics(pred):
    """
    Compute the metrics for the model. The metrics are accuracy, recall, precision, f1, and neighborhood accuracy.

    Parameters
    ----------
    pred : object
        The predictions of the model, wrapped in an object by Hugging Face trainer.

    Returns
    -------
    dict
        The dictionary of the metrics.
    """
    labels = pred.label_ids
    preds = pred.predictions

    return _compute_metrics(preds, labels)

def _compute_metrics(preds, labels):
    threshold = 0.5
    target_class = numpy.sum(labels > threshold, axis=-1) + 1
    output_class = numpy.sum(preds > threshold, axis=-1) + 1

    # compute accuracy, recall, precision, f1 for threshold 0.5
    accuracy = accuracy_score(target_class, output_class)
    recall = recall_score(target_class, output_class, average='macro', zero_division=0)
    precision = precision_score(target_class, output_class, average='macro', zero_division=0)
    f1 = f1_score(target_class, output_class, average='macro', zero_division=0)

    # compute neighborhood accuracy -- consider accurate all predictions that are off by 1
    neighborhood_accuracy = numpy.sum(
        numpy.abs(target_class - output_class) <= 1).item() / (len(labels) * 1.0)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "neighborhood_accuracy": neighborhood_accuracy
    }


def get_embedding(model, encoding):
    """
    Get the embedding from the model.

    Parameters
    ----------
    model : torch.nn.Module
        The model to get the embeddings from.
    encoding : dict
        The encoding of the text.

    Returns
    -------
    tensor
        The tensor of the embeddings.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = torch.tensor(encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(encoding["attention_mask"]).to(device)

    batch_size = 10
    model = model.to(device)
    model.eval()
    embeddings = []
    
    start_time = tm()

    with torch.no_grad():
        for i in range(0, len(input_ids), batch_size):
            batch_input_ids = input_ids[i:i+batch_size]
            batch_attention_mask = attention_mask[i:i+batch_size]
            embedding = model(batch_input_ids, attention_mask=batch_attention_mask).to("cpu")
            embeddings.append(embedding)
            gc.collect()
            torch.cuda.empty_cache()
            elapsed_time = int(tm() - start_time)
            time_per_iter = round(elapsed_time / (i+batch_size), 2)
            remaining_time = int((len(input_ids)-i)*time_per_iter)
            print(f"Status {i+batch_size}/{len(input_ids)}  Elapsed: {elapsed_time}s ({time_per_iter}s/it)  Remaining: {remaining_time}s", end="\r")
    print("Embeddings done.")
    del input_ids
    del attention_mask
    gc.collect()
    torch.cuda.empty_cache()
    embeddings = torch.cat(embeddings)
    return embeddings

def save_embedding(embeddings, filename):
    """
    Save the embeddings to a file.

    Parameters
    ----------
    embeddings : tensor
        The tensor of the embeddings.
    filename : str
        The filename to save the embeddings to.
    """
    with open(filename, 'wb') as f:
        pkl.dump(embeddings, f)

def load_embedding(filename):
    """
    Load the embeddings from a file.

    Parameters
    ----------
    filename : str
        The filename to load the embeddings from.

    Returns
    -------
    tensor
        The tensor of the embeddings.
    """
    with open(filename, 'rb') as f:
        embeddings = pkl.load(f)
    return embeddings

def prepare_dataset(embeddings, labels, num_classes=5, max_label=3500):
    labels_tensor = convert_label_to_one_hot_encodings(labels, num_classes, max_label=max_label)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    labels_tensor = labels_tensor.to(device)
    embeddings = embeddings.to(device)
    dataset = Dataset.from_dict(
        {
        "embeddings": embeddings,
        "labels": labels_tensor
        })
    return dataset

def prepare_finetune_dataset(transformer_name, texts, labels, num_classes=5, max_label=3500):
    labels_tensor = convert_label_to_one_hot_encodings(labels, num_classes, max_label=max_label)
    
    tokenizer = transformers_classes[transformer_name]["tokenizer_class"].from_pretrained(transformer_name)
    embeddings = tokenizer(texts, truncation=True, padding=True)
    
    dataset = Dataset.from_dict(
        {
        "input_ids": embeddings["input_ids"],
        "attention_mask": embeddings["attention_mask"],
        "labels": labels_tensor
        })
    return dataset

def _compute_full_embeddings(transformer_name, texts, set_name, base_path):
    
    tokenizer = transformers_classes[transformer_name]["tokenizer_class"].from_pretrained(transformer_name)
    model = transformers_classes[transformer_name]["model_class"].from_pretrained(transformer_name)

    train_encodings = tokenizer(texts, truncation=True, padding=True)

    embedding_model = EmbeddingBigBirdModel(model.bert)
    embedding_model.eval()

    train_embeddings = get_embedding(embedding_model, train_encodings)

    transformer_name_path = transformer_name.replace("/", "_")
    save_embedding(train_embeddings.cpu().numpy(), f"{base_path}/{transformer_name_path}_{set_name}_embeddings.pkl")
    return train_embeddings

def maybe_load_embeddings(transformer_name, texts, set_name, base_path):
    transformer_name_path = transformer_name.replace("/", "_")
    embeddings_filename = f"{base_path}/{transformer_name_path}_{set_name}_embeddings.pkl"
    
    if os.path.exists(embeddings_filename):
        embeddings = load_embedding(embeddings_filename)
        embeddings = torch.tensor(embeddings)
    else:
        embeddings = _compute_full_embeddings(transformer_name, texts, set_name, base_path)
    return embeddings

def get_model_class_from_name(name):
    if name == "OurClassifierModel":
        return OurClassifierModel
    elif name == "OrdinalRegressionClassifier":
        return OrdinalRegressionClassifier
    else:
        return None

In [ ]:
train_data = pd.read_csv('./data/AMT10/AMT10_train.csv')
val_data = pd.read_csv('./data/AMT10/AMT10_validation.csv')
test_data = pd.read_csv('./data/AMT10/AMT10_test.csv')

train_texts, train_labels = train_data['description'].tolist(), train_data['rating'].tolist()
val_texts, val_labels = val_data['description'].tolist(), val_data['rating'].tolist()
test_texts, test_labels = test_data['description'].tolist(), test_data['rating'].tolist()

In [ ]:
def train_and_finetune(transformer_name, train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, intermediate_layers, num_classes):
    base_model = transformers_classes[transformer_name]["model_class"].from_pretrained(transformer_name)
    embedding_size = transformers_classes[transformer_name]["embedding_size"]
    
    train_dataset=prepare_finetune_dataset(transformer_name, train_texts, train_labels, num_classes=num_classes)
    val_dataset=prepare_finetune_dataset(transformer_name, val_texts, val_labels, num_classes=num_classes)
    test_dataset=prepare_finetune_dataset(transformer_name, test_texts, test_labels, num_classes=num_classes)

    model = OurClassifierModel(base_model.bert, embedding_size, num_classes, intermediate_layers)


    training_args = TrainingArguments(
        learning_rate=1e-4,
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./logs",
        metric_for_best_model="neighborhood_accuracy"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluate the model on the testing set
    eval_results = trainer.evaluate(test_dataset)
    print(eval_results)

    transformer_name_path = transformer_name.replace("/", "_")
    torch.save(trainer.model.state_dict(), f"./models/{transformer_name_path}_classifier.pth")
    return eval_results

In [ ]:
def train_grid_search(transformer_name, train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, metric_compare_grid_results, param_grid):
  train_embeddings = maybe_load_embeddings(transformer_name, train_texts, "train", "./data/AMT10")
  val_embeddings = maybe_load_embeddings(transformer_name, val_texts, "val", "./data/AMT10")
  test_embeddings = maybe_load_embeddings(transformer_name, test_texts, "test", "./data/AMT10")

  param_combinations = list(itertools.product(*param_grid.values()))
  embedding_size = transformers_classes[transformer_name]["embedding_size"]

  best_eval_metric = float('-inf')
  best_params = None
  best_trainer = None

  for params in param_combinations:
      params = dict(zip(param_grid.keys(), params))
      learning_rate = params['learning_rate']
      train_batch_size = params['per_device_train_batch_size']
      num_epochs = params['num_train_epochs']
      model_class = get_model_class_from_name(params['model_class'])
      num_classes = params['num_classes']
      intermediate_layers = params['intermediate_layers']
      train_metric = params['train_metric']
      
      to_train_model = model_class(embeddings_size=embedding_size, num_classes=num_classes, intermediate_layers=intermediate_layers)

      train_dataset=prepare_dataset(train_embeddings, train_labels, num_classes=num_classes)
      val_dataset=prepare_dataset(val_embeddings, val_labels, num_classes=num_classes)
      test_dataset=prepare_dataset(test_embeddings, test_labels, num_classes=num_classes)


      training_args = TrainingArguments(
      learning_rate=learning_rate,
      output_dir="./results",
      num_train_epochs=num_epochs,
      per_device_train_batch_size=train_batch_size,
      per_device_eval_batch_size=train_batch_size,
      warmup_steps=500,
      weight_decay=0.01,
      logging_dir="./logs",
      metric_for_best_model=train_metric
      )

      trainer = Trainer(
          model=to_train_model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          compute_metrics=compute_metrics
      )

      trainer.train()

      eval_results = trainer.evaluate(test_dataset)

      eval_metric = eval_results[f'eval_{metric_compare_grid_results}']

      if eval_metric > best_eval_metric:
          best_eval_metric = eval_metric
          best_params = params
          best_trainer = trainer

  print(f"Best Evaluation Metric: {metric_compare_grid_results}: {best_eval_metric}")
  print(f"Best Hyperparameters: {best_params}")

  # Evaluate the model on the testing set
  test_dataset=prepare_dataset(test_embeddings, test_labels, num_classes=best_params['num_classes'])
  best_eval_results = best_trainer.evaluate(test_dataset)
  print(best_eval_results)


  transformer_name_path = transformer_name.replace("/", "_")
  torch.save(best_trainer.model.state_dict(), f"./models/{transformer_name_path}_ordinal_regression_model.pth")
  return best_params, best_eval_results

In [ ]:

param_grid = {
    'learning_rate': [
        1e-4,
    ],
    'per_device_train_batch_size': [
        256 # could be bigger (1024, 2048)
    ],
    'num_train_epochs': [
        200, # could be bigger (500, 1000)
    ],
    "train_metric": [
        "neighborhood_accuracy",
        "accuracy",
    ],

    # Model
    'model_class': [
        'OrdinalRegressionClassifier',
    ],
    'num_classes': [
        5,
        10,
        35
    ],
    'intermediate_layers': [
        [256, 128, 64, 32],
        [512, 128, 32],
        [256, 64, 16],
    ],
}

transformer_name = transformers_repo[0] # google/bigbird-roberta-base

best_params, best_eval_results = train_grid_search(transformer_name, train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, "neighborhood_accuracy", param_grid)

In [ ]:
full_model_eval_results = train_and_finetune(transformer_name, train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, best_params['intermediate_layers'], best_params['num_classes'])

In [ ]:
# TODO: load models saved on disk and evaluate them on the test set, post training